# Implementation of the GCN Classificator

#### Import useful libraries

In [1]:
import pickle
import torch
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

import torch.nn as nn
import torch.optim as optim
import torch_geometric.nn as gnn
import torch_geometric.transforms
import torch_geometric.utils
import torch.nn.functional as F
from torch_geometric.utils import train_test_split_edges
import torch_geometric as pyg

In [2]:
dtype = torch.float
has_gpu = torch.cuda.is_available()
has_mps = getattr(torch,'has_mps',False)
device = "mps" if getattr(torch,'has_mps',False) \
    else "gpu" if torch.cuda.is_available() else "cpu"

print(f"Using device: {device}")

Using device: mps


In [3]:
device = 'cpu'

#### PyG Dataset and Dataloader

In [4]:
# Load the PyG graph from ../data/graphs/pg_graph.pkl

with open('../data/graphs/pg_graph.pkl', 'rb') as f:
    pg_graph = pickle.load(f)

In [5]:
pg_graph

Data(edge_index=[2, 1090673], super_class=[49339], class=[49339], hemilineage=[49339], x=[49339, 53], edge_attr=[1090673, 6])

In [6]:
from typing import Callable, Optional
import random

import torch

from torch_geometric.data import Data, InMemoryDataset


class ConnectomeData(InMemoryDataset):
    def __init__(self, transform=None):
        super().__init__('.', transform, None, None)

        # Load your connectome data and create the Data object
        x = pg_graph.x
        edge_index = pg_graph.edge_index
        edge_attr = pg_graph.edge_attr
        y = pg_graph.super_class # for the moment only consider the super_class

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

        split = torch_geometric.transforms.RandomNodeSplit(
            num_val=0.1, num_test=0.2, key="y"
        )

        data = split(data)

        self.data, self.slices = self.collate([data])

    def __getitem__(self, idx):
        return self.data

    def __len__(self):
        return 1

dataset = ConnectomeData()

print(dataset.data)
data = dataset.data
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}') # note we are only considering 'class'
print(f'Number of nodes: {data.num_nodes}')
        

Data(x=[49339, 53], edge_index=[2, 1090673], edge_attr=[1090673, 6], y=[49339], train_mask=[49339], val_mask=[49339], test_mask=[49339])
Dataset: ConnectomeData():
Number of graphs: 1
Number of features: 53
Number of classes: 8
Number of nodes: 49339


/Users/ernestobocini/miniconda3/envs/epfl-network-ml/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [7]:
dataset.data

Data(x=[49339, 53], edge_index=[2, 1090673], edge_attr=[1090673, 6], y=[49339], train_mask=[49339], val_mask=[49339], test_mask=[49339])

#### Define the GCN architecture 

In [7]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class MyGCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(123)

        self.conv1 = GCNConv(dataset.num_features, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 32)
        self.classifier = Linear(32, dataset.num_classes)
    
    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = self.conv3(x, edge_index).relu()

        out = self.classifier(x)
        
        return out

In [8]:
gcn = MyGCN().to(device)
print(gcn)

optimizer = optim.Adam(gcn.parameters(), lr=0.1)

loss_fn = nn.CrossEntropyLoss().to(device)

MyGCN(
  (conv1): GCNConv(53, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 32)
  (classifier): Linear(in_features=32, out_features=8, bias=True)
)


In [9]:
criterion = torch.nn.CrossEntropyLoss().to(device)

In [10]:
def train(model, graph, optimizer, criterion, num_epochs, target, device = device):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(graph)
        loss = criterion(out[graph.train_mask], graph[target][graph.train_mask])
        loss.backward()
        optimizer.step()
        acc = get_accuracy(model, graph, graph.train_mask, target)

        if epoch % 10 == 0:
            print(
                f"Epoch {epoch} | Train loss {loss.item():.4f} | Train accuracy {acc:.4f}"
            )
    return model


def get_accuracy(model, graph, mask, target):
    model.eval()
    pred = model(graph).argmax(dim=1)
    return (pred[mask] == graph[target][mask]).sum() / mask.sum()

In [11]:
train(gcn, dataset.data.to(device), optimizer, loss_fn, 100, "y", device)

/Users/ernestobocini/miniconda3/envs/epfl-network-ml/lib/python3.10/site-packages/torch_geometric/utils/loop.py:370: UserWarning: MPS: nonzero op is supported natively starting from macOS 13.0. Falling back on CPU. This may have performance implications. (Triggered internally at /Users/runner/miniforge3/conda-bld/pytorch-recipe_1680607596266/work/aten/src/ATen/native/mps/operations/Indexing.mm:218.)
  edge_index = torch.cat([edge_index[:, mask], loop_index], dim=1)


Epoch 0 | Train loss 15696526336.0000 | Train accuracy 0.0020
Epoch 10 | Train loss 232091088.0000 | Train accuracy 0.1098
Epoch 20 | Train loss 1.3018 | Train accuracy 0.6529
Epoch 30 | Train loss 1.1790 | Train accuracy 0.6529
Epoch 40 | Train loss 1.1333 | Train accuracy 0.6529
Epoch 50 | Train loss 1.1183 | Train accuracy 0.6529
Epoch 60 | Train loss 1.1092 | Train accuracy 0.6529
Epoch 70 | Train loss 1.1040 | Train accuracy 0.6529
Epoch 80 | Train loss 1.1007 | Train accuracy 0.6529
Epoch 90 | Train loss 1.0979 | Train accuracy 0.6529


MyGCN(
  (conv1): GCNConv(53, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 32)
  (classifier): Linear(in_features=32, out_features=8, bias=True)
)

In [ ]:
# this remains constant because it starts predicting everything as the same class (1)

#### Evaluation of the GCN on test set


In [12]:
# let's try to predict the class of the nodes in the test set
gcn.eval()
pred = gcn(dataset.data.to(device)).argmax(dim=1)
print(pred[dataset.data.test_mask])

# let's see how many nodes we got right 
print((pred[dataset.data.test_mask] == dataset.data.y[dataset.data.test_mask]).sum() / dataset.data.test_mask.sum())


tensor([1, 1, 1,  ..., 1, 1, 1], device='mps:0')
tensor(0.6579, device='mps:0')


In [13]:
a = pred[dataset.data.test_mask].cpu().numpy()
b = pred[dataset.data.train_mask].cpu().numpy()

In [14]:
sum(a != 1), sum(b != 1)

(0, 0)

In [16]:
print(sum(dataset.data.y == 0))
print(sum(dataset.data.y == 1))
print(sum(dataset.data.y == 2))
print(sum(dataset.data.y == 3))
print(sum(dataset.data.y == 4))
print(sum(dataset.data.y == 5))
print(sum(dataset.data.y == 6))
print(sum(dataset.data.y == 7))

print(32264/len(dataset.data.y))


/Users/ernestobocini/miniconda3/envs/epfl-network-ml/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


tensor(69, device='mps:0')
tensor(32264, device='mps:0')
tensor(7800, device='mps:0')
tensor(5383, device='mps:0')
tensor(1974, device='mps:0')
tensor(1269, device='mps:0')
tensor(480, device='mps:0')
tensor(100, device='mps:0')
0.6539248870062223


## Let's try a different approach: Graph Attention Methods (GATs)

In [8]:
class BasicGraphModel(nn.Module):

    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.graphconv1 = graphnn.GATv2Conv(input_size, hidden_size)
        self.graphconv2 = graphnn.GATv2Conv(hidden_size, hidden_size)
        self.graphconv3 = graphnn.GATv2Conv(hidden_size, hidden_size)
        self.graphconv4 = graphnn.GATv2Conv(hidden_size, output_size)
        
        self.relu = nn.LeakyReLU()
      
    def forward(self, x, edge_index):

        ####### YOUR ANSWER #######
        x = self.graphconv1(x, edge_index)
        x = self.relu(x)
        x = self.graphconv2(x, edge_index)
        x = self.relu(x)
        x = self.graphconv3(x, edge_index)
        x = self.relu(x)
        x = self.graphconv4(x, edge_index)
        return x

In [57]:
data.y.numpy().argmax(1)

array([3, 1, 1, ..., 2, 5, 5])

In [47]:
a = torch.empty(3,3).normal_(mean=0,std=1)
a[range(0,3),torch.tensor([1, 0, 2]) ] = 1
a

tensor([[ 0.3669,  1.0000,  0.2130],
        [ 1.0000,  2.0537, -0.7717],
        [-0.8292,  1.5968,  1.0000]])

In [75]:
#####################################################
############## TRAIN FUNCTION #######################
#####################################################

# import the f1-score
from sklearn.metrics import f1_score

import torch_geometric.nn as graphnn

def evaluate(model, criterion, device, data):

    score_list_batch = []

    output = model(data.x, data.edge_index)
    loss_test = criterion(output[data.train_mask], data.y[data.train_mask])
    
    # creat prediction vector predict of shape (len(data.y),8)
    # predict[i,j] = 1 if the model predicts that node i is of class j
    # predict[i,j] = 0 otherwise
    predict = torch.zeros(len(data.y),8)
    predict[range(0,len(data.y)),output.argmax(1)] = 1
    
    # compute the f1-score
    score = f1_score(data.y[data.train_mask].numpy(), predict[data.train_mask], average="micro")
    return score


def train(model, criterion, device, optimizer, max_epochs, data):

    epoch_list = []
    scores_list = []

    # loop over epochs
    for epoch in range(max_epochs):
        model.train()
        
        optimizer.zero_grad()
        # logits is the output of the model
        logits = model(data.x, data.edge_index).to(device)
        # compute the loss
        loss = criterion(logits[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        print("Epoch {:05d} | Loss: {:.4f}".format(epoch , loss.item()))
        

        if epoch % 10 == 0:
            # evaluate the model on the validation set
            # computes the f1-score (see next function)
            score = evaluate(model, criterion, device, data)
            print("F1-Score: {:.4f}".format(score))
            scores_list.append(score)
            epoch_list.append(epoch)

    return epoch_list, scores_list

Let's train the model

In [72]:
from torch_geometric.utils import one_hot
dataset.data.y = one_hot(dataset.data.y, num_classes=8)

ValueError: 'index' tensor needs to be one-dimensional

In [73]:
dataset.data

Data(x=[49339, 53], edge_index=[2, 1090673], edge_attr=[1090673, 6], y=[49339, 8], train_mask=[49339], val_mask=[49339], test_mask=[49339])

In [76]:
### Max number of epochs
max_epochs = 200

### DEFINE THE MODEL
basic_model = BasicGraphModel(  input_size = dataset.num_features, 
                                hidden_size = 256, 
                                output_size = dataset.num_classes).to(device)

### DEFINE LOSS FUNCTION
criterion = nn.BCEWithLogitsLoss()

### DEFINE OPTIMIZER
optimizer = torch.optim.Adam(basic_model.parameters(), lr=0.005)

### TRAIN THE MODEL
epoch_list, basic_model_scores = train(basic_model, criterion, device, optimizer, max_epochs, dataset.data)

/Users/ernestobocini/miniconda3/envs/epfl-network-ml/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Epoch 00001 | Loss: 28842885120.0000
Accuracy: 0.6534
Epoch 00002 | Loss: 8636674048.0000
Epoch 00003 | Loss: 1622594688.0000
Epoch 00004 | Loss: 2275024128.0000
Epoch 00005 | Loss: 2876649728.0000
Epoch 00006 | Loss: 25421989888.0000
Epoch 00007 | Loss: 28595822592.0000
Epoch 00008 | Loss: 51094102016.0000
Epoch 00009 | Loss: 32293806080.0000
Epoch 00010 | Loss: 4376730624.0000
Epoch 00011 | Loss: 3996340224.0000
Accuracy: 0.1092
Epoch 00012 | Loss: 3868876288.0000
Epoch 00013 | Loss: 4366283776.0000
Epoch 00014 | Loss: 4838290944.0000
Epoch 00015 | Loss: 5311051264.0000
Epoch 00016 | Loss: 5663306240.0000
Epoch 00017 | Loss: 5900253184.0000
Epoch 00018 | Loss: 5989053440.0000
Epoch 00019 | Loss: 6077882880.0000
Epoch 00020 | Loss: 19925850112.0000
Epoch 00021 | Loss: 33167579136.0000
Accuracy: 0.0014
Epoch 00022 | Loss: 41463767040.0000
Epoch 00023 | Loss: 44394823680.0000
Epoch 00024 | Loss: 49299562496.0000
Epoch 00025 | Loss: 65092026368.0000
Epoch 00026 | Loss: 77869662208.0000
E

KeyboardInterrupt: 

In [26]:
### F1-SCORE ON TEST DATASET
score_test = evaluate(basic_model, loss_fcn, device, test_dataloader)
print("Basic Model : F1-Score on the test set: {:.4f}".format(score_test))

### PLOT EVOLUTION OF F1-SCORE W.R.T EPOCHS
def plot_f1_score(epoch_list, scores) :
    plt.figure(figsize=[10,5])
    plt.plot(epoch_list, scores)
    plt.title("Evolution of F1S-Score w.r.t epochs")
    plt.ylim([0.0, 1.0])
    plt.show()
    
plot_f1_score(epoch_list, basic_model_scores)

NameError: name 'loss_fcn' is not defined